In [1]:
import pandas as pd
import re


In [4]:
# Load the Excel file with the specific tab
file_path = '../data/Combined Data.xlsx'
tab_name = 'TBL_Nielsen_Unit_Sales_Extract'

In [5]:
df_SKU = pd.read_excel(file_path, sheet_name=tab_name)
print(f'Successfully loaded {tab_name} from {file_path}')

Successfully loaded TBL_Nielsen_Unit_Sales_Extract from ../data/Combined Data.xlsx


In [22]:
# Helper function to clean pack size
def clean_pack_size(pack_size):
        if pd.isna(pack_size):
            return None
        
        # Convert to string
        pack_size = str(pack_size).strip().upper()
        
        # Extract numbers using regex
        numbers = re.findall(r'\d+\.?\d*', pack_size)
        if numbers:
            return float(numbers[0])
        return None

def clean_unit_size(unit_size):
        if pd.isna(unit_size):
            return None
        
        if unit_size == '330ML':
            return 0.33
        elif unit_size == '500ML':
            return 0.5
        elif unit_size == '300ML-400ML':
            return 0.35
       

In [23]:
 # Create new cleaned pack size column
df_SKU['PackSizeCleaned'] = df_SKU['PACK SIZE'].apply(clean_pack_size)
df_SKU['UnitSizeCleaned'] = df_SKU['UNIT SIZE'].apply(clean_unit_size)

In [24]:
# Filter for dates in 2025
df_SKU['DATE'] = pd.to_datetime(df_SKU['DATE'])
df_SKU_2025 = df_SKU[df_SKU['DATE'].dt.year == 2025].copy()

# Create aggregations
agg_functions = {
    'Sales Price per Unit': 'mean',
    'Sales Units': 'sum'
}

df_aggregated = df_SKU_2025.groupby(['ITEM', 'Level_1', 'Level_3','PackSizeCleaned','UnitSizeCleaned']).agg(agg_functions).reset_index()

# Rename columns for clarity
df_aggregated = df_aggregated.rename(columns={
    'Sales Price per Unit': 'Average_Price_per_Unit',
    'Sales Units': 'Total_Sales_Units'
})

print(df_aggregated)

                                                 ITEM      Level_1  \
0                       DB DOUBLE BROWN CANS 18X330ML   TOTAL BEER   
1                         DB DRAUGHT BOTTLES 24X330ML   TOTAL BEER   
2                       DB EXPORT 33 BOTTLES 12X330ML   TOTAL BEER   
3                       DB EXPORT 33 BOTTLES 15X330ML   TOTAL BEER   
4                       DB EXPORT 33 BOTTLES 24X330ML   TOTAL BEER   
5                   DB EXPORT CITRUS BOTTLES 12X330ML   TOTAL BEER   
6                     DB EXPORT GOLD BOTTLES 12X330ML   TOTAL BEER   
7                     DB EXPORT GOLD BOTTLES 15X330ML   TOTAL BEER   
8                     DB EXPORT GOLD BOTTLES 24X330ML   TOTAL BEER   
9                        DB EXPORT GOLD CANS 12X330ML   TOTAL BEER   
10          DB EXPORT ULTRA LOW CARB BOTTLES 12X330ML   TOTAL BEER   
11          DB EXPORT ULTRA LOW CARB BOTTLES 24X330ML   TOTAL BEER   
12             DB EXPORT ULTRA LOW CARB CANS 24X330ML   TOTAL BEER   
13                  

In [36]:
specific_item = 'DB DOUBLE BROWN CANS 18X330ML'
item_data = df_SKU_2025[df_SKU_2025['ITEM'] == specific_item]
print(item_data['Sales Units'])
print(item_data['Sales Units'].sum())

103      468.0
104      403.0
105      623.0
106      580.0
107      467.0
         ...  
47423    902.0
47424    825.0
47425    842.0
47426    852.0
47427    865.0
Name: Sales Units, Length: 65, dtype: float64
57517.64


In [46]:
# Calculate volume
df_aggregated['VolumePerUnit'] = df_aggregated['PackSizeCleaned'] * df_aggregated['UnitSizeCleaned']
df_aggregated['CustomerPrice'] = df_aggregated['Average_Price_per_Unit']
df_aggregated['VolumeSold'] = df_aggregated['PackSizeCleaned'] * df_aggregated['UnitSizeCleaned'] * df_aggregated['Total_Sales_Units']
df_aggregated['PriceElasticity'] = -1.5

df_aggregated['PricePerL'] = df_aggregated['CustomerPrice'] / df_aggregated['VolumePerUnit']
df_aggregated['GrossProfit'] = df_aggregated['PricePerL'] * .60 * 100
df_aggregated['RSV'] = df_aggregated['PricePerL'] * df_aggregated['VolumeSold'] / 1000000
df_aggregated['Profit'] = df_aggregated['GrossProfit'] * df_aggregated['VolumeSold'] / 100 / 1000000

# Calculate total volume sold
total_volume = df_aggregated['VolumeSold'].sum()
total_value = df_aggregated['RSV'].sum()

# Calculate VoMS
df_aggregated['VoMS'] = df_aggregated['VolumeSold'] / total_volume * 100
df_aggregated['VaMS'] = df_aggregated['RSV'] / total_value * 100


print(df_aggregated)


                                                 ITEM      Level_1  \
0                       DB DOUBLE BROWN CANS 18X330ML   TOTAL BEER   
1                         DB DRAUGHT BOTTLES 24X330ML   TOTAL BEER   
2                       DB EXPORT 33 BOTTLES 12X330ML   TOTAL BEER   
3                       DB EXPORT 33 BOTTLES 15X330ML   TOTAL BEER   
4                       DB EXPORT 33 BOTTLES 24X330ML   TOTAL BEER   
5                   DB EXPORT CITRUS BOTTLES 12X330ML   TOTAL BEER   
6                     DB EXPORT GOLD BOTTLES 12X330ML   TOTAL BEER   
7                     DB EXPORT GOLD BOTTLES 15X330ML   TOTAL BEER   
8                     DB EXPORT GOLD BOTTLES 24X330ML   TOTAL BEER   
9                        DB EXPORT GOLD CANS 12X330ML   TOTAL BEER   
10          DB EXPORT ULTRA LOW CARB BOTTLES 12X330ML   TOTAL BEER   
11          DB EXPORT ULTRA LOW CARB BOTTLES 24X330ML   TOTAL BEER   
12             DB EXPORT ULTRA LOW CARB CANS 24X330ML   TOTAL BEER   
13                  

In [47]:
# Filter for dates in 2025
df_aggregated['OWNERSHIP'] = 'OWN'

# Save aggregated data
output_path = '../data/aggregated_sales_2025.csv'
df_aggregated.to_csv(output_path, index=False)
print(f'\nData exported successfully to: {output_path}')


Data exported successfully to: ../data/aggregated_sales_2025.csv
